In [1]:
import logging
import os

from dotenv import load_dotenv
from typing import Union, List

load_dotenv()
print(os.environ["TRANSFORMERS_VERBOSITY"])

info


In [2]:
import torch
# from huggingface_hub import login
from transformers import (
    logging as tlogging,
    pipeline,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

# --- Enable transformers / HF logging ---
logging.basicConfig(level=logging.INFO)
tlogging.set_verbosity_info()  # transformers logger

print(f"os.environ['TRANSFORMERS_VERBOSITY']= {os.environ['TRANSFORMERS_VERBOSITY']}")
print(f"tlogging.get_verbosity_info() set to: {logging.getLevelName(tlogging.get_verbosity())}")

/Users/prakashyenugandula/github/yprakash/llm_red_teaming/phi3.5-hallucination/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


os.environ['TRANSFORMERS_VERBOSITY']= info
tlogging.get_verbosity_info() set to: INFO


In [3]:
def device_specific_values():
    device, dtype = None, None
    if torch.cuda.is_available():
        print("GPU is available to use")
        device = torch.device("cuda")
        dtype = torch.bfloat16

    elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        print("MPS is available to use")
        device = torch.device("mps")
        dtype = torch.float16      # safest for Apple MPS (bfloat16 has partial support)
    else:
        print("Using CPU, as GPU/MPS is NOT available")
        device = torch.device("cpu")
        dtype = torch.float32      # CPU fallback for numerical stability

    return device, dtype

device, dtype = device_specific_values()
print(f"{dtype}, {device}")

MPS is available to use
torch.float16, mps


In [4]:
model_id = "grounded-ai/phi3.5-hallucination-judge"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,               # Reduce weights to 4 bits
    bnb_4bit_quant_type="nf4",       # Normalized float quantization
    bnb_4bit_use_double_quant=True,  # Secondary quantization for stability
)
bnb_config

PackageNotFoundError: No package metadata was found for bitsandbytes

In [7]:
model_id = "grounded-ai/phi3.5-hallucination-judge"
#print(dtype)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",  # Auto-place layers (CPU/MPS)
    # quantization_config=bnb_config,
)

print(model)

loading file tokenizer.model from cache at /Users/prakashyenugandula/.cache/huggingface/hub/models--grounded-ai--phi3.5-hallucination-judge/snapshots/5d3630986c814c7c0651ba5e0b4a71c7e66cc021/tokenizer.model
loading file tokenizer.json from cache at /Users/prakashyenugandula/.cache/huggingface/hub/models--grounded-ai--phi3.5-hallucination-judge/snapshots/5d3630986c814c7c0651ba5e0b4a71c7e66cc021/tokenizer.json
loading file added_tokens.json from cache at /Users/prakashyenugandula/.cache/huggingface/hub/models--grounded-ai--phi3.5-hallucination-judge/snapshots/5d3630986c814c7c0651ba5e0b4a71c7e66cc021/added_tokens.json
loading file special_tokens_map.json from cache at /Users/prakashyenugandula/.cache/huggingface/hub/models--grounded-ai--phi3.5-hallucination-judge/snapshots/5d3630986c814c7c0651ba5e0b4a71c7e66cc021/special_tokens_map.json
loading file tokenizer_config.json from cache at /Users/prakashyenugandula/.cache/huggingface/hub/models--grounded-ai--phi3.5-hallucination-judge/snapshot

OSError: grounded-ai/phi3.5-hallucination-judge does not appear to have a file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt or flax_model.msgpack.